# import

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ES_HOST = os.getenv("ES_HOST")
ES_USER_ID = os.getenv("ES_ID")
ES_PASSWD = os.getenv("ES_PWD")
ES_INDEX_NAME = os.getenv("ES_INDEX_NAME")
EMBEDDING_BASE_URL=os.getenv("EMBEDDING_BASE_URL")
EMBEDDING_MODEL=os.getenv("EMBEDDING_MODEL")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGUNGFACE_TOKEN")

## Define tools

In [2]:
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch
from langchain_community.embeddings import HuggingFaceHubEmbeddings
from langchain_openai import ChatOpenAI
from pydantic.v1 import BaseModel, Field  # <-- Uses v1 namespace
from langchain.tools import StructuredTool

In [3]:

class SearchToolInput(BaseModel):
    """Input for the index show data tool."""

    index_name: str = Field(
        ..., description="The name of the index for which the data is to be retrieved. in this case indexname will be " + ES_INDEX_NAME
    )
    query: str = Field(..., description="The ElasticSearch JSON query used to filter all hits. Should use the _source field if possible to specify required fields.")
    from_: int = Field(
        ..., description="The record index from which the query will start"
    )
    size: int = Field(
        ...,
        description="How many records will be retrieved from the ElasticSearch query",
    )

In [4]:
embeddings = HuggingFaceHubEmbeddings(model=EMBEDDING_BASE_URL, huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN)
es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
vector_store = ElasticsearchStore(
    embedding=embeddings,
    index_name=ES_INDEX_NAME,
    es_connection=es
)
retriever = vector_store.as_retriever()

search_results = retriever.invoke("openai")
print(search_results[0])
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.

page_content='오픈AI, 10대와 가족 위한 교육 자료 개발
생성형 인공지능(AI) 시대를 연 오픈AI가 10대와 가족들을 위한 AI 교육 자료를 만들고 있다고 밝혔다.
29일(현지 시각) CNBC 등에 따르면 오픈AI는 10대들이 안전하게 AI를 사용할 수 있도록 '커먼 센스 미디어'와 파트너십을 맺었다고 밝혔다.
샘 올트먼 오픈AI 최고경영자(CEO)는 "우리는 이 도구를 교육 경험의 일부로 사용할 청소년과 사람들이 안전하고 책임감 있게 사용할 수 있는 방법을 찾고 싶다"라고 밝혔다.
이 글자크기로 변경됩니다.
(예시) 가장 빠른 뉴스가 있고 다양한 정보, 쌍방향 소통이 숨쉬는 다음뉴스를 만나보세요. 다음뉴스는 국내외 주요이슈와 실시간 속보, 문화생활 및 다양한 분야의 뉴스를 입체적으로 전달하고 있습니다.
29일(현지 시각) CNBC 등에 따르면 오픈AI는 10대들이 안전하게 AI를 사용할 수 있도록 ‘커먼 센스 미디어’와 파트너십을 맺었다고 밝혔다. 샘 올트먼 오픈AI 최고경영자(CEO)는 “우리는 이 도구를 교육 경험의 일부로 사용할 청소년과 사람들이 안전하고 책임감 있게 사용할 수 있는 방법을 찾고 싶다”라고 밝혔다.
커먼 세스는 아이들이 기술을 안전하게 사용하고 접근할 수 있도록 하는 데 중점을 둔 비영리단체다. 오픈AI와 커먼 센스는 어린이, 교육자, 부모를 위한 AI 지침서와 교육자료를 만드는 데 있다. 커먼센스 미디어의 CEO 짐 스테이어는 “가족과 교육자에게 챗GPT의 안전하고 책임감 있는 사용에 대해 교육하고 이 새로운 기술이 의도하지 않은 결과를 피할 수 있도록 설계할 것”이라고 설명했다.' metadata={'id': 336431, 'hash_key': '515af3aa67d7a9504c65d632fce82c04', 'title': '오픈AI, 10대와 가족 위한 교육 자료 개발', 'created_date': '2024-01-30T10:03:04', 'portal': 'daum', 'media': '매일경제', 'url': 

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpointEmbeddings`.
  warn_deprecated(
/var/folders/jy/5_rl6575533638lskdr9s6s00000gn/T/ipykernel_46471/3231746536.py:2: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch([ES_HOST], basic_auth=(ES_USER_ID, ES_PASSWD), ca_certs="./../certs/ca.crt", verify_certs=False, timeout=120)
/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://192.168.0.10:9200' using TLS with verify_certs=False is insecure
  _tran

In [5]:
def create_es_search_tool():
      return StructuredTool(name="elastic_index_search_tool",
                            func=retriever.get_relevant_documents, 
                            args_schema=SearchToolInput)
tools = [create_es_search_tool()]

## define llm

In [6]:
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

In [7]:
from langchain_core.messages import HumanMessage

response = llm.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

### agentic

In [8]:
model_with_tools = llm.bind_tools(tools)

In [9]:
response = model_with_tools.invoke([HumanMessage(content=f"{ES_INDEX_NAME} 색인에서 삼성전자 뉴스를 찾아줘!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'elastic_index_search_tool', 'args': {'index_name': 'news_article_embedding', 'query': '{"query": {"match": {"content": "삼성전자"}}}', 'from_': 0, 'size': 5}, 'id': 'call_64YzpffPWATw8muffix2b0fb', 'type': 'tool_call'}]


# Create Agent

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content=f"2024년 삼성전자에 대한 부정적인 뉴스를 찾아서 하나 하나 부정적인 지수를 점수로 매겨서 리스트로 표시해")]}
)
response["messages"]

/Users/nuri/dev/jupyter/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.0.10'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[HumanMessage(content='2024년 삼성전자에 대한 부정적인 뉴스를 찾아서 하나 하나 부정적인 지수를 점수로 매겨서 리스트로 표시해', id='4fca0d9b-d464-4830-ac1a-a935696346c2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UBh0oK45j07Q0PZ2CGEge9NM', 'function': {'arguments': '{"index_name": "news_article_embedding", "query": "{\\"query\\":{\\"bool\\":{\\"must\\":[{\\"match\\":{\\"content\\":\\"삼성전자\\"}},{\\"match\\":{\\"content\\":\\"부정\\"}}]}}}", "from_": 0, "size": 5}', 'name': 'elastic_index_search_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 163, 'total_tokens': 241}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_400f27fa1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-21b096e5-a9b1-4f2c-9e4d-0a637549da14-0', tool_calls=[{'name': 'elastic_index_search_tool', 'args': {'index_name': 'news_article_embedding', 'query': '{"query":{"bool":{"must":[{"match":{"content":"삼성전자"}},{"match":{"content":"부정"}}]}}}', 'from_': 0, 

In [12]:
response["messages"][-1]

AIMessage(content='다음은 2024년 삼성전자에 대한 부정적인 뉴스 기사와 그에 대한 부정적인 지수 점수입니다.\n\n### 1. [비즈&] 삼성전자, 글로벌 전략회의…\'복합위기 대응책\' 모색 外\n- **작성일**: 2023-12-14\n- **매체**: 연합뉴스TV\n- **URL**: [기사 링크](https://n.news.naver.com/mnews/article/422/0000634426?rc=N&ntype=RANKING&sid=101)\n- **부정적인 지수**: 5/10\n- **요약**: 삼성전자가 글로벌 전략회의를 통해 복합위기 대응책을 모색하고 있다는 내용으로, 위기 상황을 언급함.\n\n### 2. [속보]삼성전자, 4분기 영업익 2.8조…전년비 35%↓\n- **작성일**: 2024-01-09\n- **매체**: 뉴시스\n- **URL**: [기사 링크](https://n.news.naver.com/mnews/article/003/0012307841?rc=N&ntype=RANKING&sid=101)\n- **부정적인 지수**: 8/10\n- **요약**: 삼성전자의 4분기 영업이익이 전년 대비 35% 감소한 2.8조 원이라는 소식으로, 상당한 수익 감소를 다루고 있음.\n\n### 3. [사진] 네이버·삼성전자 합작 ‘AI 반도체’\n- **작성일**: 2023-12-20\n- **매체**: 중앙일보\n- **URL**: [기사 링크](https://n.news.naver.com/mnews/article/025/0003329676?rc=N&ntype=RANKING&sid=105)\n- **부정적인 지수**: 2/10\n- **요약**: 네이버와 삼성전자가 합작하여 AI 반도체를 개발한다는 긍정적인 내용으로, 부정적인 요소는 적음.\n\n### 4. [속보] 삼성전자, 외신보도 즉각 반박 "HBM 공급 위한 테스트 진행 중"\n- **작성일**: 2024-05-24\n- **매체**: 한경비즈니스\n- **URL**: [

1. **기사 제목:** "메모리 수요 회복·재고 정상화…1Q 흑자전환 예상" - 삼성전자 컨콜  
**작성일자:** 2024-01-31
**부정적인 지수:** 3.5
**기사 링크:** [네이버](https://n.news.naver.com/mnews/article/018/0005664707?rc=N&ntype=RANKING&sid=101)
**요약:** 메모리 수요 회복과 재고 정상화에 따라 1분기 흑자전환이 예상되지만, 전년 대비 실적이 부진할 것으로 전망됨.

2. **기사 제목:** "[하나證 주간추천주]우리금융지주·삼성전기·엘앤에프"
**작성일자:** 2024-03-30
**부정적인 지수:** 4.0
**기사 링크:** [다음](https://v.daum.net/v/20240330124950413)
**요약:** 삼성전기는 1분기 잠정실적 발표 예정이며, 메모리 업황 호조에도 주가 순자산비율이 평균 수준에 못 미침.

3. **기사 제목:** "[속보]삼성전자, 4분기 영업이익 2.8조…전년비 35%↓"
**작성일자:** 2024-01-09
**부정적인 지수:** 4.5
**기사 링크:** [네이버](https://n.news.naver.com/mnews/article/003/0012307841?rc=N&ntype=RANKING&sid=101)
**요약:** 삼성전자의 4분기 영업이익이 전년 대비 35% 감소한 2.8조 원으로 집계됨.

4. **기사 제목:** "[컨콜]삼성전자 "1분기 메모리 사업 흑자전환 예상""
**작성일자:** 2024-01-31
**부정적인 지수:** 3.0
**기사 링크:** [네이버](https://n.news.naver.com/mnews/article/277/0005374162?rc=N&ntype=RANKING&sid=101)
**요약:** 메모리 사업의 1분기 흑자전환이 예상되나, 전반적인 실적은 부진할 것으로 예상됨.

5. **기사 제목:** "[속보]삼성전자 작년 영업이익 6조5400억 원…전년비 85%↓"
**작성일자:** 2024-01-09
**부정적인 지수:** 4.5
**기사 링크:** [네이버](https://n.news.naver.com/mnews/article/421/0007278353?rc=N&ntype=RANKING&sid=101)
**요약:** 삼성전자의 작년 영업이익이 전년 대비 85% 감소한 6조5400억 원으로 집계됨.

6. **기사 제목:** "D램·낸드, 올해 상반기에도 선별적 생산조정" - 삼성전자 컨콜
**작성일자:** 2024-01-31
**부정적인 지수:** 3.5
**기사 링크:** [네이버](https://n.news.naver.com/mnews/article/018/0005664737?rc=N&ntype=RANKING&sid=101)
**요약:** D램과 낸드플래시의 생산 조정이 상반기에도 지속될 예정이며, 이는 전체적인 실적에 부정적인 영향을 미칠 것으로 보임.

각 기사의 부정적인 지수는 1에서 5 사이로, 5에 가까울수록 부정적인 수준이 높습니다.',